In [130]:
import psycopg2
from configparser import ConfigParser
import pandas as pd

import common

In [131]:
csv_save_path="C:\\Users\\shrus\\Documents\\Synthetic-data-generation\\sampled_data_csv_1000\\"

In [132]:
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Connect to postgresql

In [133]:
def postgresql_config(filename='config.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [134]:
def test_postgresql_connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = postgresql_config()

        print('------Test Connection------')

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()

        # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)

        # close the communication with the PostgreSQL
        cur.close()
        print('Database connection closed.')
        # Successfully connected, return True
        print('Successfully connected!')
        return True
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
        print('------Test Finished------')

In [135]:
test_postgresql_connect()


------Test Connection------
Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.3, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.
Successfully connected!
------Test Finished------


True

### Querying Patient Data from PostgreSQL

In [136]:
def get_patients():
    """ Query data from the patients table """
    conn = None
    try:
        params = postgresql_config()
        conn = psycopg2.connect(**params)
        cur = conn.cursor()

        cur.execute("SELECT * from patients")
        print("The number of parts: ", cur.rowcount)
        row = cur.fetchone()

        while row is not None:
            print(row)
            row = cur.fetchone()

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [137]:
# Select some patients as sample
patient_num = 100

sql = "SELECT * FROM mimiciii.patients p order by random() limit " + str(patient_num)
conn = psycopg2.connect(**postgresql_config())

In [138]:
# Execute the sql and form the result as DataFrame
random_patients_df = pd.read_sql(sql, conn)

In [139]:
random_patients_df

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
0,43164,87197,F,2054-10-12,NaT,NaT,NaT,0
1,1671,1765,M,2183-05-09,NaT,NaT,NaT,0
2,20072,21269,M,2078-07-25,2162-12-10,2162-12-10,2162-12-10,1
3,5388,5692,M,2099-09-01,NaT,NaT,NaT,0
4,28302,30096,F,2080-05-03,2161-10-08,NaT,2161-10-08,1
...,...,...,...,...,...,...,...,...
95,35742,58667,M,2067-06-29,2130-08-20,2130-08-20,2130-08-20,1
96,3212,3391,F,2143-03-01,NaT,NaT,NaT,0
97,29722,31650,M,2090-11-19,NaT,NaT,NaT,0
98,15026,15895,M,2039-10-21,2124-07-11,NaT,2124-07-11,1


In [140]:
# Save patient sample
random_patients_df.to_csv(csv_save_path + "patients.csv", index=None)

In [141]:
# Pick Subject_id
patients_subject_ids = random_patients_df['subject_id']
patients_subject_ids

0     87197
1      1765
2     21269
3      5692
4     30096
      ...  
95    58667
96     3391
97    31650
98    15895
99    52058
Name: subject_id, Length: 100, dtype: int64

In [142]:
random_patients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   row_id       100 non-null    int64         
 1   subject_id   100 non-null    int64         
 2   gender       100 non-null    object        
 3   dob          100 non-null    datetime64[ns]
 4   dod          28 non-null     datetime64[ns]
 5   dod_hosp     20 non-null     datetime64[ns]
 6   dod_ssn      25 non-null     datetime64[ns]
 7   expire_flag  100 non-null    int64         
dtypes: datetime64[ns](4), int64(3), object(1)
memory usage: 6.4+ KB


In [143]:
# create query scripts

def create_query_scripts(table_name):
    return "SELECT * FROM " + table_name + " WHERE subject_id = "

In [144]:
#Query data from database
def select_data(subject_ids, sql):
    '''
    Query data according to the subject from table
    :param subject_ids:
    :param sql:
    :return:
    '''

    # Create the connection object
    conn = None

    try:
        # Get connection
        params = postgresql_config()
        conn = psycopg2.connect(**params)
        
        # Create a empty DataFrame
        table_df = pd.DataFrame()
        # Iterate subject_ids and query data about patients
        for subject_id in subject_ids:
            new_sql = sql + str(subject_id)
            # Query data and transform to DataFrame form
            select_result = pd.read_sql(new_sql, conn)

            # The first cycle, initial the DataFrame
            if table_df.empty:
                table_df = select_result.copy()
            # DataFrame was initialed, just append the data
            else:
                table_df = pd.concat([table_df, select_result], axis=0)

        return table_df
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [145]:
def query_others(patients_subject_ids, table_name_list):
    #Pass in a list containing table names, traverse the list, and query data from each table
    #:param patients_subject_ids: patients id
    #:param table_name_list: table list

    for table_name in table_name_list:
        print("Start querying: " + table_name)
        # Query data
        sample = select_data(patients_subject_ids, create_query_scripts(table_name))
        # Save sample to csv file
        sample.to_csv(csv_save_path + table_name + ".csv", index=None)
    print("All done!" )

In [146]:
table_list = ['mimiciii.admissions', 'mimiciii.callout', 'mimiciii.icustays', 'mimiciii.chartevents', 'mimiciii.outputevents', 'mimiciii.inputevents_mv', 
            'mimiciii.procedureevents_mv', 'mimiciii.cptevents', 'mimiciii.labevents', 'mimiciii.microbiologyevents', 'mimiciii.procedures_icd','mimiciii.prescriptions']

In [147]:
query_others(patients_subject_ids, table_list)


Start querying: mimiciii.admissions
Start querying: mimiciii.callout
Start querying: mimiciii.icustays
Start querying: mimiciii.chartevents
Start querying: mimiciii.outputevents
Start querying: mimiciii.inputevents_mv
Start querying: mimiciii.procedureevents_mv
Start querying: mimiciii.cptevents
Start querying: mimiciii.labevents
Start querying: mimiciii.microbiologyevents
Start querying: mimiciii.procedures_icd
Start querying: mimiciii.prescriptions
All done!
